In [2]:
import mne
import os
from mne.preprocessing import (create_eog_epochs, create_ecg_epochs,
                               compute_proj_ecg, compute_proj_eog)
import utils.variables as var
import scipy.io as sio

In [ ]:
class Filtering:
    '''
    A Python class that handles all of the filtering of data, including saving the filtered data to new .mat-files
    '''
    root         = '/Users/idaandreassen/Desktop/MASTER/stress_detection_EEG/Data'
    dir_raw      = root + var.DIR_RAW
    dir_filtered = root + var.DIR_FILTERED
    dir_psd      = root + var.DIR_PSD

    sfreq = var.SFREQ
    ch_type = 'eeg'
    n_channels = var.NUM_CHANNELS

    def __init__(self, sub_nr, ses_nr, run_nr):
        self.sub_nr = sub_nr
        self.ses_nr = ses_nr
        self.run_nr = run_nr

        self.load_data(self)

        #Create MNE RawArray
        info = mne.create_info(8, sfreq=self.sfreq, ch_types=self.ch_type, verbose=None)
        self.raw_arr = mne.io.RawArray(self.data, info)
        
        mne.rename_channels(self.raw_arr.info, var.MAPPING)



    #-----------------------------------------------FUNCTIONS-----------------------------------------

    def load_data(self):
        dir      = self.dir_raw
        data_key = 'raw_eeg_data'

        # Load one recording
        filename  = f"/sub-{self.sub_nr}_ses-{self.ses_nr}_run-{self.run_nr}.mat"
        f         = dir + filename
        self.data = sio.loadmat(f)[data_key]

    def save_data(self):
        